In [119]:
# %%
import os
from locale import setlocale
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from numpy import sin,cos,arcsin,sqrt,abs,pi,log10,exp
from scipy.fftpack import fft,ifft
from scipy.io import wavfile
from scipy.stats import gmean 
from tqdm import tqdm
from compress_pickle import dump as cpkldump # reading/writing compressed pickles
from compress_pickle import load as cpklload # reading/writing compressed pickles

#obspy
from obspy import UTCDateTime
#from obspy.core.trace import max

from obspy.taup import TauPyModel
model = TauPyModel(model="iasp91")

#rtergpy
from rtergpy.run import defaults, event, etime2name, src2ergs
from rtergpy.waveforms import getwaves, get_respinv

#attenuation
from AttenuationFunctions import processANSS, freqmaxes


# Processing and Reading information about event stored in ANSS_data.txt

Defaults = defaults()
Event = event()
Defaults.src="RASPISHAKE"
Defaults.network="AM"
Defaults.chan="EHZ"
Defaults.stationrange=[1.,10.]
Event.ecount='00'
Event.iter='RS'
# Event.newData = False   # use already downloaded data
Event.newData=False
edateold=""

processANSS() #process to remove unneccesary information
# ANSS = pd.read_csv('ANSS_processed_data.csv', sep=',', comment='#')
ANSS = pd.read_csv('ANSS_processed_data.csv')
#print(ANSS) #Just to check if processing correctly

# run everything above to test on command line
for index, EQ in ANSS.iterrows():
    network = "AM"
    chan = "EHZ"
    src = "RASPISHAKE"
    rads = [1.,10.]
    eloc = [EQ.Latitude,EQ.Longitude,EQ.Depth] 
    MagType = [EQ.Mtype]
    MagValue = [EQ.Mag]
    Magnitude = [MagType, MagValue]
    year,mo,dy = EQ.Date.split('-')
    hh,mn,sec = EQ.Time.split(':')
    etime=(UTCDateTime(int(year),int(mo),int(dy),int(hh),int(mn),float(sec)))
    
    if EQ.Date == edateold:
        Event.ecount=str(int(Event.ecount)+1).zfill(2)
    else:
        Event.ecount='00'
    edateold=EQ.Date
    Event.eventname=etime2name(etime,ecount=Event.ecount)
    Event.origin=[eloc,etime]

    print("\n\n"+Event.eventname+" ===============================")
    try:
        st, df = [], []
        st, df = getwaves(Defaults=Defaults,Event=Event)
    except:
        print("ERROR: running on "+Event.eventname+" failed!!!!\n\n")
               
# make an inventory incase needed. 

inventory = get_respinv(network,eloc,etime,rads,chan,src)

ImportError: cannot import name 'max' from 'obspy.core.trace' (/Users/hkunwer/opt/anaconda3/envs/rtergpy/lib/python3.11/site-packages/obspy/core/trace.py)

In [6]:
# filter for instrument response and taper

taper=0.05

stp = st.copy()  # create backup

# process data (maybe make this into a function)

stp.detrend(type='polynomial', order=5) # pre-instrument removal
stp.taper(taper)
stp.remove_response(output="DISP")
stp.detrend(type='polynomial', order=5) # post-instrument removal
stp.taper(taper)

# process each individual frequency band in a bandpass using .filter
# frequency bands of 0.1-0.25, 0.25-0.5, 0.5-0.75, 0.75-1, 1-1.5

#freqs = ([0.1,0.25],[0.25,0.5],[0.50,0.75],[0.75,1],[1,1.25])

# freq1 band = 0.1-0.25,
stp_freq1 = stp.copy()
stp_freq1.filter("bandpass",freqmin=0.10 , freqmax=0.25)

# freq2 band = 0.25-0.5,
stp_freq2 = stp.copy()
stp_freq2.filter("bandpass", freqmin=0.25, freqmax=0.50)

# # freq3 band = 0.5-0.75,
stp_freq3 = stp.copy()
stp_freq3.filter("bandpass", freqmin=0.50, freqmax=0.75)

# # freq4 band = 0.75-1.
stp_freq4 = stp.copy()
stp_freq4.filter("bandpass", freqmin=0.75, freqmax=1.0)

# # freq5 band = 1-1.25
stp_freq5 = stp.copy()
stp_freq5.filter("bandpass", freqmin=1.0, freqmax=1.25)

# #add all freq to a tuple
stp_freqs = (stp_freq1,stp_freq2,stp_freq3,stp_freq4,stp_freq5)

# # Plot all the wiggles, shows how frequency filtering works. 

# #stp[0].plot(); #only instrument response removal
# #stp_freq1[0].plot(); # bandpass filter of 0.1-0.25
# #stp_freq2[0].plot(); # bandpass filter of 0.25-0.5
# #stp_freq3[0].plot(); # bandpass filter of 0.5-0.75
# #stp_freq4[0].plot(); # bandpass filter of 0.75-0.1
# #stp_freq5[0].plot(); # bandpass filter of 1-1.25

In [120]:
# # %%
# # use freqmaxes function to get maxamps for all traces in one frequency band
# # compare the plots formed from the function above with previous tests in Appendix below

#freqs = ([0.1,0.25],[0.25,0.5],[0.50,0.75],[0.75,1],[1,1.25])

maxamps, dist_str = maximumAmplitude(stp_freq)
       
df = pd.DataFrame({"maxamps":maxamps,"distance":dist_str}) 

TypeError: 'numpy.float64' object is not iterable

In [109]:
df

,maxamps,distance
0,0.000079,115.294131


In [105]:
#plt.xlabel("Distance (km)", fontsize =12)
#plt.ylabel("Max Amplitude (log)", fontsize = 12)
#plt.title("Maximum Amplitude vs. Distance for each Frequency Band")
#plt.legend()
#plt.grid(True) 
#plt.show()
#plt.savefig('first_plot.png', dpi=300) #also maybe loop so it'll iterate and name them better for continuous storage

In [116]:
# filter for instrument response and taper

taper=0.05

stp = st.copy()  # create backup

# process data (maybe make this into a function)

stp.detrend(type='polynomial', order=5) # pre-instrument removal
stp.taper(taper)
stp.remove_response(output="DISP")
stp.detrend(type='polynomial', order=5) # post-instrument removal
stp.taper(taper)

# process each individual frequency band in a bandpass using .filter
# frequency bands of 0.1-0.25, 0.25-0.5, 0.5-0.75, 0.75-1, 1-1.5

#freqs = ([0.1,0.25],[0.25,0.5],[0.50,0.75],[0.75,1],[1,1.25])

#change this to read in input and make a list of all the bandpass turn this into a function
# that saves all this for each frequency band by a different name

frequencymin = 0.1
frequencymax = 0.25

stp_freq = stp.copy()
stp_freq.filter("bandpass",freqmin=frequencymin , freqmax= frequencymax)

stp_freqs = (stp_freq)
#stp_freq[0].plot();



In [117]:
# # %%
# # use freqmaxes function to get maxamps for all traces in one frequency band
# # compare the plots formed from the function above with previous tests in Appendix below

#freqs = ([0.1,0.25],[0.25,0.5],[0.50,0.75],[0.75,1],[1,1.25])

maxamps = []
dist = []
ALLmaxamps = [] 
dist_str = []

for stpf in stp_freq:

    for tr in stpf:
        maxamps = tr.max
        dist = tr.stats.distance / 1000
        ALLmaxamps.append(maxamps)
        dist_str.append(dist)
       
df = pd.DataFrame({"maxamps":ALLmaxamps,"distance":dist_str}) 

TypeError: 'numpy.float64' object is not iterable